#1. File upload

model_name = 'res10_300x300_ssd_iter_140000.caffemodel'

prototxt_name = 'deploy.prototxt.txt'

file_name = 'obama_01.mp4'

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!ls -al

#2 Load File

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import IPython

In [ ]:
model_name = 'res10_300x300_ssd_iter_140000.caffemodel'
prototxt_name = 'deploy.prototxt.txt'
file_name = 'obama_01.mp4'

In [ ]:
min_confidence = 0.3
frame_width = 300
frame_height = 300

#3. DNN Model

In [ ]:
model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)

In [ ]:
cap = cv2.VideoCapture(file_name)

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("size: {0} x {1}".format(width, height))

# 영상 저장을 위한 VideoWriter 인스턴스 생성
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('test.avi', fourcc, 24, (int(width), int(height)))

In [ ]:
def detectAndDisplay(frame):
    IPython.display.clear_output(wait=True)
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (frame_width, frame_height)), 1.0, (frame_width, frame_height), (104.0, 177.0, 123.0))
    model.setInput(blob)
    detections = model.forward()
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability)
        confidence = detections[0, 0, i, 2]
        if confidence > min_confidence:
            (height, width) = frame.shape[:2]
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = box.astype('int')
            print(confidence, startX, startY, endX, endY)
            text = '{:.2f}%'.format(confidence*100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0,255,0), 2)
            cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_COMPLEX, 0.5 , (0, 255, 0), 1)
    #Show the output
    cv2_imshow(frame)

In [ ]:
#-- 2. Read the video stream
cap = cv2.VideoCapture(file_name)

if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break    
    detectAndDisplay(frame)
     writer.write(frame)

cap.release()
writer.release()  # 저장 종료

In [ ]:
def download(path):
    try:
        from google.colab import files
        files.download(path)
    except ImportError:
        import os
        print('Error download: ', os.path.join(os, getcwd(), path))

In [ ]:
download('test.avi')